# **IMPORT LIBRARIES AND LOAD DATA**

In [ ]:
pip install wrds --no-deps # install WRDS package

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import wrds
from google.colab import files

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Connect to WRDS**

In [ ]:
# establish a connection to the WRDS (input username & password)
db = wrds.Connection()
db.create_pgpass_file()

Enter your WRDS username [root]:gusti
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


# **DATA COLLECTION**

## **1. Select 25 Top Stocks**

In [ ]:
# get the earliest trading date for each permno
query_earliest_date = """
SELECT
    permno,
    MIN(date) as first_trade_date
FROM
    crsp.dsf
GROUP BY
    permno
HAVING
    MIN(date) <= '2005-01-01'
"""

earliest_dates = db.raw_sql(query_earliest_date)

# ensure stocks are still active until December 29, 2023 (latest available date)
query_active_stocks = """
SELECT
    permno
FROM
    crsp.dsf
WHERE
    date = '2023-12-29'
GROUP BY
    permno
"""

active_stocks = db.raw_sql(query_active_stocks)

# combine the two sets of stocks to get those listed before 2005 and still active in 2023
filtered_permnos = earliest_dates.merge(active_stocks, on='permno', how='inner')
# exclude 3M (misclassified) & HUM (no news data in IQ KeyDev)
filtered_permnos = filtered_permnos.loc[~filtered_permnos['permno'].isin([22592, 48653])]

# get the list of permnos as a comma-separated string
permnos_str = ','.join([str(permno) for permno in filtered_permnos['permno'].tolist()])

# get market capitalisation, company name, and sector information for healthcare sector
query_main = f"""
SELECT
    a.permco,
    a.permno,
    a.date,
    a.shrout,
    a.prc * a.shrout as market_cap,
    b.shrcd,
    b.exchcd,
    b.siccd,
    b.ncusip,
    b.comnam
FROM
    crsp.dsf AS a
JOIN
    crsp.dsenames AS b
ON
    a.permno = b.permno
WHERE
    ((b.siccd BETWEEN 2830 AND 2836) OR
    (b.siccd BETWEEN 3841 AND 3843) OR
    b.siccd = 3826 OR
    b.siccd = 3829 OR
    b.siccd = 5047 OR
    b.siccd = 5122 OR
    b.siccd = 5912 OR
    b.siccd = 6324 OR
    b.siccd = 8071 OR
    b.siccd = 8072 OR
    b.siccd = 8092 OR
    b.siccd = 8093 OR
    b.siccd = 8099)
    AND a.permno IN ({permnos_str})
    AND a.date = '2023-12-29'
    AND b.exchcd IN (1, 3)
"""

# execute query
crsp_data = db.raw_sql(query_main)

In [ ]:
# filter data for the latest date
latest_date = crsp_data['date'].max()
latest_data = crsp_data[crsp_data['date'] == latest_date]

# group by permco and permno and select the entry with the highest market capitalisation within each group
top_25_healthcare_stocks = latest_data.groupby(['permco', 'permno']).apply(lambda x: x.nlargest(1, 'market_cap'))

# sort by market capitalization and get the top 25 stocks
top_25_healthcare_stocks = top_25_healthcare_stocks.sort_values(by='market_cap', ascending=False).head(25)
top_25_healthcare_stocks.reset_index(drop=True, inplace=True)

In [ ]:
# save the dataframe
top25_path = '/content/drive/MyDrive/PostGrad/5. Extended Research Projects/Dataset/top25healthcare.csv'

top_25_healthcare_stocks.to_csv(top25_path, index=False)

## **2. Collect Price and Return Data**

In [ ]:
# get permno of the top 25 stocks
top_25_permnos = top_25_healthcare_stocks['permno'].tolist()

# convert permno list to a string for the SQL IN clause
permnos_str = ', '.join(map(str, top_25_permnos))

In [ ]:
# download training data set

# define the date range
start_date = '2005-01-01'
end_date = '2015-12-31'

# query to get data for the specified date range and variables for the top 25 stocks
query = f"""
SELECT
    a.permco,
    a.permno,
    b.comnam,
    b.ticker,
    a.date,
    a.prc,
    a.cfacpr,
    a.ret
FROM
    crsp.dsf AS a
JOIN
    (SELECT permno, comnam, ticker, namedt, nameendt
     FROM crsp.dsenames
     WHERE permno IN ({permnos_str}) -- filter for the top 25 stocks
       AND namedt <= '{end_date}'
       AND (nameendt IS NULL OR nameendt >= '{start_date}')) AS b
ON
    a.permno = b.permno
WHERE
    a.permno IN ({permnos_str})     -- filter for the top 25 stocks
    AND a.date BETWEEN '{start_date}' AND '{end_date}'
    AND a.date >= b.namedt
    AND (a.date <= b.nameendt OR b.nameendt IS NULL)
"""

# execute query
crsp_train = db.raw_sql(query)
crsp_train.sort_values(by=['permco', 'date'], inplace=True)

In [ ]:
# check missing values
crsp_train[crsp_train.isna().any(axis=1)]

,permco,permno,comnam,ticker,date,prc,cfacpr,ret
41391,10876,76614,REGENERON PHARMACEUTICALS INC,REGN,2015-06-09,NaN,1.0,NaN
45895,11042,76744,VERTEX PHARMACEUTICALS INC,VRTX,2011-04-28,NaN,1.0,NaN
46910,11042,76744,VERTEX PHARMACEUTICALS INC,VRTX,2015-05-12,NaN,1.0,NaN


In [ ]:
# download test data set

# define the date range
start_date = '2016-01-01'
end_date = '2023-12-31'

# query to get data for the specified date range and variables for the top 25 stocks
query = f"""
SELECT
    a.permco,
    a.permno,
    b.comnam,
    b.ticker,
    a.date,
    a.prc,
    a.cfacpr,
    a.ret
FROM
    crsp.dsf AS a
JOIN
    (SELECT permno, comnam, ticker, namedt, nameendt
     FROM crsp.dsenames
     WHERE permno IN ({permnos_str}) -- filter for the top 25 stocks
       AND namedt <= '{end_date}'
       AND (nameendt IS NULL OR nameendt >= '{start_date}')) AS b
ON
    a.permno = b.permno
WHERE
    a.permno IN ({permnos_str})       -- filter for the top 25 stocks
    AND a.date BETWEEN '{start_date}' AND '{end_date}'
    AND a.date >= b.namedt
    AND (a.date <= b.nameendt OR b.nameendt IS NULL)
"""

# execute query
crsp_test = db.raw_sql(query)
crsp_test.sort_values(by=['permco', 'date'], inplace=True)

In [ ]:
# change ticker for permco 42273 to ELV for consistency
crsp_train['ticker'][crsp_train['permco']==42273] = 'ELV'
crsp_test['ticker'][crsp_test['permco']==42273] = 'ELV'

<ipython-input-11-d44926cf61db>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_train['ticker'][crsp_train['permco']==42273] = 'ELV'
<ipython-input-11-d44926cf61db>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_test['ticker'][crsp_test['permco']==42273] = 'ELV'


In [ ]:
# save the dataframe
crsp_train_path = '/content/drive/MyDrive/PostGrad/5. Extended Research Projects/Dataset/crsp_train.csv'
crsp_test_path = '/content/drive/MyDrive/PostGrad/5. Extended Research Projects/Dataset/crsp_test.csv'

crsp_train.to_csv(crsp_train_path, index=False)
crsp_test.to_csv(crsp_test_path, index=False)

## **3. Retrieve Linking Table**

In [ ]:
# query to get linking table of gvkey and permco
query = f"""
SELECT
    DISTINCT lpermno,
    lpermco,
    gvkey,
    conm
FROM
    crsp.ccm_lookup
WHERE
    lpermno IN ({permnos_str}) -- filter top 25 stocks
"""

# execute query
linking = db.raw_sql(query)

In [ ]:
# save the dataframe
linking_path = '/content/drive/MyDrive/PostGrad/5. Extended Research Projects/Dataset/linking_table.csv'

linking.to_csv(linking_path, index=False)

## **4. Collect News Data**

In [ ]:
# extract top 25 gvkey list
top_25_gvkey = linking['gvkey'].tolist()

# convert gvkey list to a string for the SQL IN clause
gvkey_str = ', '.join(f"'{gv}'" for gv in top_25_gvkey)

In [ ]:
# download training data set

# define the date range (a week early than numerical data)
start_date = '2004-12-27'
end_date = '2015-12-27'

# query to get news data for the specified date range and variables for the top 25 stocks
query = f"""
SELECT
    a.companyname,
    a.headline,
    b.situation,
    a.eventtype,
    a.announcedate,
    a.gvkey
FROM
    ciq_keydev.wrds_keydev AS a
LEFT JOIN
    (SELECT keydevid, situation
     FROM ciq_keydev.ciqkeydev
     WHERE announceddate BETWEEN '{start_date}' AND '{end_date}') AS b
ON
    a.keydevid = b.keydevid
WHERE
    a.announcedate BETWEEN '{start_date}' AND '{end_date}'
    AND a.gvkey IN ({gvkey_str})
"""

# execute query
kd_train = db.raw_sql(query)

In [ ]:
# download test data set

# define the date range (a week early than numerical data)
start_date = '2015-12-28'
end_date = '2023-12-24'

# query to get news data for the specified date range and variables for the top 25 stocks
query = f"""
SELECT
    a.companyname,
    a.headline,
    b.situation,
    a.eventtype,
    a.announcedate,
    a.gvkey
FROM
    ciq_keydev.wrds_keydev AS a
LEFT JOIN
    (SELECT keydevid, situation
     FROM ciq_keydev.ciqkeydev
     WHERE announceddate BETWEEN '{start_date}' AND '{end_date}') AS b
ON
    a.keydevid = b.keydevid
WHERE
    a.announcedate BETWEEN '{start_date}' AND '{end_date}'
    AND a.gvkey IN ({gvkey_str})
"""

# execute query
kd_test = db.raw_sql(query)

In [ ]:
# save the dataframe
kd_train_path = '/content/drive/MyDrive/PostGrad/5. Extended Research Projects/Dataset/kd_train.csv'
kd_test_path = '/content/drive/MyDrive/PostGrad/5. Extended Research Projects/Dataset/kd_test.csv'

kd_train.to_csv(kd_train_path, index=False)
kd_test.to_csv(kd_test_path, index=False)

## **5. Collect S&P500 Market Return (for portfolio analysis)**

In [ ]:
# query to get S&P500 return data from CRSP
query = """
    SELECT caldt, sprtrn AS sp500_return
    FROM crsp.dsp500
    WHERE caldt >= '2005-01-01' AND caldt <= '2023-12-31'
    ORDER BY caldt
"""

# execute query
spx500_returns = db.raw_sql(query)

## **6. Collect Market Cap for Top 25 Healthcare Stocks (for Portfolio Analysis)**

In [ ]:
# query to get market cap overtime (2005-2023) for 25 healthcare stocks
query = f"""
SELECT
    permco,
    date,
    prc * shrout as market_cap
FROM
    crsp.dsf
WHERE
    permno IN ({permnos_str})   -- filter top 25 stocks
    AND date >= '2005-01-01' AND date <= '2023-12-31'
ORDER BY date
"""

# execute query
market_cap = db.raw_sql(query)

## **7. Collect NYSE Market Cap (for Transaction Cost)**

In [ ]:
# Query to get market cap overtime (2005-2023) for all stocks
query = f"""
SELECT
    permco,
    date,
    prc * shrout as market_cap
FROM
    crsp.dsf
WHERE
    date >= '2005-01-01' AND date <= '2023-12-31'
    AND hexcd = 1  -- Filter for NYSE stocks
ORDER BY date
"""

# Execute the query
market_cap_all = db.raw_sql(query)

In [ ]:
# close the connection
db.close()

# **DATA PREPROCESSING**

## **1. Preprocessing Price and Return Data**

### **Training Dataset**

**A. Calculate Adjusted Price**

In [ ]:
# format to datetime
crsp_train['date'] = pd.to_datetime(crsp_train['date'], format='%Y-%m-%d')

In [ ]:
# define function to calculate adjusted price
def calculate_adjusted_price(group):
  group['adj_prc'] = group['prc']/ group['cfacpr']
  return group

# calculate adjusted price
crsp_train = crsp_train.groupby('permco').apply(calculate_adjusted_price).reset_index(drop=True)

**B. Calculate Weekly Return**

In [ ]:
# filter out rows with null or negative price and return
crsp_train_filtered = crsp_train[(crsp_train['prc'] > 0) & ~crsp_train['ret'].isna()]
crsp_train_filtered.set_index('date', inplace=True)

# define function to calculate weekly return and set the weekly adjusted price at the end of the week
def calculate_weekly_return(x):
    weekly_return = (np.exp(np.sum(np.log(1 + x['ret']))) - 1)
    adj_prc = x['adj_prc'].iloc[-1]
    return pd.Series({'weekly_ret': weekly_return, 'adj_prc': adj_prc})

# apply the function for each permco, grouping by week (ending on Sunday)
weekly_returns_train = crsp_train_filtered.groupby(['permco', 'ticker', pd.Grouper(freq='W-SUN')]).apply(calculate_weekly_return)
weekly_returns_train = weekly_returns_train.reset_index()

# rename the columns
weekly_returns_train.rename(columns={0: 'weekly_ret', 'date': 'end_date'}, inplace=True)

# add start_date to represent the beginning of the week
weekly_returns_train['start_date'] = weekly_returns_train['end_date'] - pd.DateOffset(days=6)

**C. Extract Price Direction**

In [ ]:
# sort to ensure correct order for price comparison
weekly_returns_train.sort_values(by=['permco', 'end_date'], inplace=True)

# calculate the change in adjusted price from the previous week for each permco
weekly_returns_train['price_change'] = weekly_returns_train.groupby(['permco', 'ticker'])['adj_prc'].diff()

# determine the direction of the price change and label it as 'positive', 'negative', or 'no change'
weekly_returns_train['price_direction'] = weekly_returns_train['price_change'].apply(
    lambda x: 'positive' if x > 0 else ('negative' if x < 0 else 'no change'))

# drop unnecessary price_change column
weekly_returns_train.drop('price_change', axis=1, inplace=True)

### **Test Dataset**

**A. Calculate Adjusted Price**

In [ ]:
# format to datetime
crsp_test['date'] = pd.to_datetime(crsp_test['date'], format='%Y-%m-%d')

In [ ]:
# define function to calculate adjusted price
def calculate_adjusted_price(group):
  group['adj_prc'] = group['prc']/ group['cfacpr']
  return group

# calculate adjusted price
crsp_test = crsp_test.groupby('permco').apply(calculate_adjusted_price).reset_index(drop=True)

**B. Calculate Weekly Return**

In [ ]:
# filter out rows with null or negative price and return
crsp_test_filtered = crsp_test[(crsp_test['prc'] > 0) & ~crsp_test['ret'].isna()]
crsp_test_filtered.set_index('date', inplace=True)

# define function to calculate weekly return and set the weekly adjusted price at the end of the week
def calculate_weekly_return(x):
    weekly_return = (np.exp(np.sum(np.log(1 + x['ret']))) - 1)
    adj_prc = x['adj_prc'].iloc[-1] #last price in the week
    return pd.Series({'weekly_ret': weekly_return, 'adj_prc': adj_prc})

# apply the function for each permco, grouping by week (ending on Sunday)
weekly_returns_test = crsp_test_filtered.groupby(['permco', 'ticker', pd.Grouper(freq='W-SUN')]).apply(calculate_weekly_return)
weekly_returns_test = weekly_returns_test.reset_index()

# rename the columns
weekly_returns_test.rename(columns={0: 'weekly_ret', 'date': 'end_date'}, inplace=True)

# add start_date to represent the beginning of the week
weekly_returns_test['start_date'] = weekly_returns_test['end_date'] - pd.DateOffset(days=6)

**C. Extract Price Direction**

In [ ]:
# sort to ensure correct order for price comparison
weekly_returns_test.sort_values(by=['permco', 'end_date'], inplace=True)

# calculate the change in adjusted price from the previous week for each permco
weekly_returns_test['price_change'] = weekly_returns_test.groupby(['permco', 'ticker'])['adj_prc'].diff()

# determine the direction of the price change and label it as 'positive', 'negative', or 'no change'
weekly_returns_test['price_direction'] = weekly_returns_test['price_change'].apply(
    lambda x: 'positive' if x > 0 else ('negative' if x < 0 else 'no change'))

# drop unnecessary price_change column
weekly_returns_test.drop('price_change', axis=1, inplace=True)

## **2. Preprocessing News Data**

### **Training Dataset**

In [ ]:
# format to datetime
kd_train['announcedate'] = pd.to_datetime(kd_train['announcedate'], format='%Y-%m-%d')

In [ ]:
# find duplicates
duplicates = kd_train.duplicated(subset=['gvkey', 'announcedate', 'headline', 'situation', 'eventtype'], keep=False)
duplicate_kd_train = kd_train[duplicates]

# remove duplicate news (only keep the last occurrence)
kd_train_unique = kd_train.drop_duplicates(subset=['gvkey', 'announcedate', 'headline', 'situation', 'eventtype'], keep='last')

### **Test Dataset**

In [ ]:
# format to datetime
kd_test['announcedate'] = pd.to_datetime(kd_test['announcedate'], format='%Y-%m-%d')

In [ ]:
#find duplicates
duplicates = kd_test.duplicated(subset=['gvkey', 'announcedate', 'headline', 'situation', 'eventtype'], keep=False)
duplicate_kd_test = kd_test[duplicates]

#remove headline duplicates (only keep the last occurrence)
kd_test_unique = kd_test.drop_duplicates(subset=['gvkey', 'announcedate', 'headline', 'situation', 'eventtype'], keep='last')

## **3. Merge Price, Return Data with News**

### **Training Dataset**

In [ ]:
# add permco as identifier in news dataset
kd_train_unique = pd.merge(kd_train_unique, linking[['lpermco', 'gvkey']], on='gvkey', how='left')
kd_train_unique = kd_train_unique.rename(columns={'lpermco':'permco'})

In [ ]:
# calculate the start and end dates of the announcement week based on the announcement date
kd_train_unique = kd_train_unique.assign(
    start_date=kd_train_unique['announcedate'] - pd.to_timedelta(kd_train_unique['announcedate'].dt.weekday, unit='d'),
    end_date=kd_train_unique['announcedate'] + pd.to_timedelta(6 - kd_train_unique['announcedate'].dt.weekday, unit='d')
)

# calculate the end date of the following week by adding 7 days to the original end date
kd_train_unique['end_date_nw'] = kd_train_unique['end_date'] + pd.DateOffset(days=7)

In [ ]:
# left join return & price data with news by date (news date is one week early)
ret_news_train = pd.merge(weekly_returns_train, kd_train_unique[['permco','end_date_nw', 'announcedate', 'headline', 'situation', 'eventtype']],
                          left_on=['permco', 'end_date'], right_on=['permco', 'end_date_nw'], how='left')

In [ ]:
# remove end_date_nw column
ret_news_train.drop('end_date_nw', axis=1, inplace=True)

# impute week with no news with no_headlines
ret_news_train.fillna('no_headlines', inplace=True)

<ipython-input-34-118f14ab0a06>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'no_headlines' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  ret_news_train.fillna('no_headlines', inplace=True)


### **Test Dataset**

In [ ]:
# add permco as identifier in news dataset
kd_test_unique = pd.merge(kd_test_unique, linking[['lpermco', 'gvkey']], on='gvkey', how='left')
kd_test_unique = kd_test_unique.rename(columns={'lpermco':'permco'})

In [ ]:
# calculate the start and end dates of the announcement week based on the announcement date
kd_test_unique = kd_test_unique.assign(
    start_date=kd_test_unique['announcedate'] - pd.to_timedelta(kd_test_unique['announcedate'].dt.weekday, unit='d'),
    end_date=kd_test_unique['announcedate'] + pd.to_timedelta(6 - kd_test_unique['announcedate'].dt.weekday, unit='d')
)

# calculate the end date of the following week by adding 7 days to the original end date
kd_test_unique['end_date_nw'] = kd_test_unique['end_date'] + pd.DateOffset(days=7)

In [ ]:
# left join return & price data with news by date (news date is one week early)
ret_news_test = pd.merge(weekly_returns_test, kd_test_unique[['permco','end_date_nw', 'announcedate', 'headline', 'situation', 'eventtype']],
                         left_on=['permco', 'end_date'], right_on=['permco', 'end_date_nw'], how='left')

In [ ]:
# remove end_date_nw column
ret_news_test.drop('end_date_nw', axis=1, inplace=True)

# impute week with no news with no_headlines
ret_news_test.fillna('no_headlines', inplace=True)

<ipython-input-38-943ad071427c>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'no_headlines' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  ret_news_test.fillna('no_headlines', inplace=True)


In [ ]:
# save the dataframe
train_path = '/content/drive/MyDrive/PostGrad/5. Extended Research Projects/Dataset/train_data.csv'
test_path = '/content/drive/MyDrive/PostGrad/5. Extended Research Projects/Dataset/test_data.csv'

ret_news_train.to_csv(train_path, index=False)
ret_news_test.to_csv(test_path, index=False)

## **4. Preprocessing Market Data (for Portfolio Analysis)**

### **S&P 500 Weekly Return**

In [ ]:
# define function to calculate weekly return for S&P 500
def calculate_weekly_return(x):
    weekly_return = (np.exp(np.sum(np.log(1 + x))) - 1)
    return weekly_return

# format to datetime
spx500_returns['caldt'] = pd.to_datetime(spx500_returns['caldt'])

# apply function to calculate weekly return
spx500_returns.set_index('caldt', inplace=True)
spx500_weekly_returns = spx500_returns['sp500_return'].resample('W-MON').apply(calculate_weekly_return)

spx500_weekly_returns = spx500_weekly_returns.reset_index()

In [ ]:
# save the dataframe
spx500_path = '/content/drive/MyDrive/PostGrad/5. Extended Research Projects/Dataset/spx500_weekly_returns.csv'

spx500_weekly_returns.to_csv(spx500_path, index=False)

### **Top 25 Healthcare Weekly Market Cap**

In [ ]:
# format to datetime
market_cap['date'] = pd.to_datetime(market_cap['date'])

# create a new column for the week start date (Monday)
market_cap['week_start_date'] = market_cap['date'] - pd.to_timedelta(market_cap['date'].dt.dayofweek, unit='d')

# group by permco and week start date, then aggregate market_cap
weekly_market_cap = market_cap.groupby(['permco', 'week_start_date']).agg({
    'market_cap': 'first'
}).reset_index()

In [ ]:
# save the dataframe
marketcap_path = '/content/drive/MyDrive/PostGrad/5. Extended Research Projects/Dataset/market_cap.csv'

weekly_market_cap.to_csv(marketcap_path, index=False)

### **NYSE Weekly Market Cap**

In [ ]:
# format to datetime
market_cap_all['date'] = pd.to_datetime(market_cap_all['date'])

# create a new column for the week start date (Monday)
market_cap_all['week_start_date'] = market_cap_all['date'] - pd.to_timedelta(market_cap_all['date'].dt.dayofweek, unit='d')

# group by permco and week start date, then aggregate market_cap_all
weekly_market_cap_all = market_cap_all.groupby(['permco', 'week_start_date']).agg({
    'market_cap': 'first'
}).reset_index()

In [ ]:
# save the dataframe
marketcapall_path = '/content/drive/MyDrive/PostGrad/5. Extended Research Projects/Dataset/market_cap_nyse.csv'

weekly_market_cap_all.to_csv(marketcapall_path, index=False)